# Exploring US Economic Data with a Bokeh Dashboard

This project intents to visualize simple time-series data in a dashboard and makes it permanently available in an S3 bucket.

1) Economic/Unemployment Data Sources: https://research.stlouisfed.org/ and https://fred.stlouisfed.org/series/UNRATE/

2) Data Packaged Core Datasets containing GDP: https://github.com/datasets/gdp-us

In [1]:
import os
import pandas as pd
import wget
from bokeh.plotting import figure, output_file, show,output_notebook

output_notebook()

Loading BokehJS ...

Load the GDP and unemployment csv's from the links dictionary into pandas data frames.

In [2]:
#dfGDP = pd.read_csv(f'{os.getcwd()}\\Data\\gdp_data.csv')
dfGDP = pd.read_csv(wget.download("https://raw.githubusercontent.com/datasets/gdp-us/master/quarter.csv",out=f'{os.getcwd()}\\Data\\US_Economic_Data'))

dfGDP['date'] = pd.to_datetime(dfGDP['date'])

dfGDP

,date,level-current,level-chained,change-current,change-chained
0,1947-04-01,246.3,1932.3,6.0,-0.8
1,1947-07-01,250.1,1930.3,17.3,6.4
2,1947-10-01,260.3,1960.7,9.6,6.2
3,1948-01-01,266.2,1989.5,10.7,6.8
4,1948-04-01,272.9,2021.9,10.1,2.3
...,...,...,...,...,...
279,2017-01-01,19057.7,16903.2,4.2,3.0
280,2017-04-01,19250.0,17031.1,4.8,2.8
281,2017-07-01,19500.6,17163.9,5.1,2.3
282,2017-10-01,19754.1,17286.5,4.3,2.2


In [3]:
# download and load csv to dataframe
dfU = pd.read_csv(wget.download("https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=UNRATE&scale=left&cosd=1948-01-01&coed=2020-08-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Quarterly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2020-09-17&revision_date=2020-09-17&nd=1948-01-01",
                                out=f'{os.getcwd()}\\Data\\US_Economic_Data'))

# rename columns
dfU.columns = ['date','unemployment']

# remove latest value, since it is just a symbol
dfU = dfU[:-1]

dfU['unemployment'] = dfU['unemployment'].astype(float)

dfU['date'] = pd.to_datetime(dfU['date'])

dfU

,date,unemployment
0,1948-01-01,3.733333
1,1948-04-01,3.666667
2,1948-07-01,3.766667
3,1948-10-01,3.833333
4,1949-01-01,4.666667
...,...,...
285,2019-04-01,3.633333
286,2019-07-01,3.633333
287,2019-10-01,3.533333
288,2020-01-01,3.833333


In [4]:
# In which years was unemployment higher than 8.5%?

dfU[dfU.unemployment> 8.5]

,date,unemployment
109,1975-04-01,8.866667
136,1982-01-01,8.833333
137,1982-04-01,9.433333
138,1982-07-01,9.900000
139,1982-10-01,10.666667
140,1983-01-01,10.366667
141,1983-04-01,10.133333
142,1983-07-01,9.366667
143,1983-10-01,8.533333
245,2009-04-01,9.300000


## Plotting an interactive graph

In [5]:
# Define a dashboard function for interactive visualization with the bokeh library

def make_dashboard(x, gdp_change, unemployment, title, file_name):
    output_file(file_name)
    p = figure(title=title, x_axis_label='year', y_axis_label='%', x_axis_type='datetime')
    p.line(x.squeeze(), gdp_change.squeeze(), color="firebrick", line_width=4, legend_label="% GDP change")
    p.line(x.squeeze(), unemployment.squeeze(), line_width=4, legend_label="% unemployed")
    show(p)

In [6]:
# For the graph we define an x-axis variable. In this case the date column from the unemployment data frame.

x = dfU['date']

In [7]:
# Create new data frame with y-axis variable 1

gdp_change = dfGDP['change-current']

In [8]:
# Create new data frame with y-axis variable 2

unemployment = dfU['unemployment']

In [9]:
# Graph title

title = 'U.S. economics: GDP vs. Unemployment Rate'

In [14]:
# Bokeh creates a html file for the graph. A file name has to be defined.

file_name = f'{os.getcwd()}\\Data\\US_Economic_Data\\index.html'

In [15]:
# Call the function for the dashboard visualization

make_dashboard(x, gdp_change, unemployment, title, file_name)

### Screenshot if dashboard does not show up
![dash title](Data/US_Economic_Data/dash.png)

### Disabled: Deploy the dashboard in S3 Object Storage

### Generate a URL to the graph url.